In [1]:
from utils.experimentor import *
from utils.constants import *


In [2]:
config = {
    "dataset_name": Dataset.OGBN_PRODUCTS,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 10000,
    'num_of_runs': 1,
    'patience_period': 40,
    
    #'should_visualize': False,
    'batch_size': 256,
    'test_batch_size': 256,
    'num_workers': 0,
    'force_cpu': False,
    'test_frequency': 5,
    'console_log_freq': 1,
    'do_train_tqdm_logging': True,
    
    'lr': 0.001,
    'num_of_layers': 3,  # PPI has got 42% of nodes with all 0 features - that's why 3 layers are useful
    'num_heads': 1,
    'hidden_size': 128,
    'dropout': 0.5,  
    "use_layer_norm": False,
    "use_saint": False,
    
    #'neighbourhood_degree': 2,
    #'adjacency_mode': AdjacencyMode.OneStep
}

In [3]:
experimentor = Experimentor(config)
experimentor.run()

torch.cuda is available: True
Used Dataset: Dataset.OGBN_PRODUCTS

Run 01:



Epoch 01: 100%|██████████| 196615/196615 [01:12<00:00, 2727.91it/s]


Epoch 01| Loss: 1.3533, Approx. Train: 0.6802


Epoch 02: 100%|██████████| 196615/196615 [00:53<00:00, 3654.79it/s]


Epoch 02| Loss: 0.7561, Approx. Train: 0.8046


...

Epoch 35: 100%|██████████| 196615/196615 [00:25<00:00, 7733.14it/s]


Epoch 35| Loss: 0.4785, Approx. Train: 0.8710


Epoch 36: 100%|██████████| 196615/196615 [00:25<00:00, 7774.67it/s]


Epoch 36| Loss: 0.4810, Approx. Train: 0.8703


Evaluating:   9%|▊         | 428032/4898058 [00:24<02:02, 36491.86it/s]